<a href="https://colab.research.google.com/github/NishiAjmera/notebooks/blob/main/PersonalAssisstant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --upgrade --quiet  duckduckgo-search langchain_community langchain_openai langchain langgraph langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import json

from langchain_openai import ChatOpenAI

from langgraph.graph import END, StateGraph
from langgraph.prebuilt.tool_node import ToolNode
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun
from langchain.pydantic_v1 import BaseModel, Field
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage


class DDGInput(BaseModel):
    query: str = Field(description="search query to look up")


class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]


class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}


prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

tools = [DuckDuckGoSearchRun(args_schema=DDGInput)]

model = ChatOpenAI(model="gpt-4o", api_key="").bind_tools(tools)

abot = Agent(model, tools, system=prompt)

In [ ]:
messages = [HumanMessage(content="what route should I take to got from canry wharf to canning town")]
result = abot.graph.invoke({"messages": messages})
print(result['messages'][-1].content)

Calling: {'name': 'duckduckgo_search', 'args': {'query': 'route from Canary Wharf to Canning Town'}, 'id': 'call_LktByM45orgrU0Itp8RDR54v'}
Back to the model!
You can take the Jubilee Line from Canary Wharf to Canning Town Station. This is a direct route and typically the quickest way to travel between these two locations.


In [ ]:
messages = [HumanMessage(content="what time is the next train from canary wharf to canning town ? Ccheck the official website or another source for the latest schedule")]
result = abot.graph.invoke({"messages": messages})
print(result['messages'][-1].content)

Calling: {'name': 'duckduckgo_search', 'args': {'query': 'next train from Canary Wharf to Canning Town schedule'}, 'id': 'call_Dfh3yqw869nmuV73sTPM5X8U'}
Back to the model!
Calling: {'name': 'duckduckgo_search', 'args': {'query': 'next Jubilee Line train from Canary Wharf to Canning Town'}, 'id': 'call_RZtFp80fSOdwf5aH3NWYM37Y'}
Back to the model!
To find the most accurate and up-to-date information for the next train from Canary Wharf to Canning Town on the Jubilee Line, it is best to check the official Transport for London (TfL) website or their live departure boards.

You can visit the [TfL website](https://tfl.gov.uk/) and use their journey planner or live departures feature for real-time information. Simply enter "Canary Wharf" as the departure station and "Canning Town" as the arrival station to see the next available trains.


In [ ]:
messages = [HumanMessage(content="yes")]
result = abot.graph.invoke({"messages": messages})
print(result['messages'][-1].content)

How can I assist you today? If you have a specific question or need information on a particular topic, please let me know!
